In [1]:
cd /lustre/scratch126/cellgen/team205/kt21/all_multiome_cellranger_arc/

/lustre/scratch126/cellgen/team205/kt21/all_multiome_cellranger_arc


In [1]:
pwd

'/home/jovyan/Scratch126'

In [2]:
import sys

import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib as mpl
import scipy

import scvi
import hashlib

from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42 # enables correct plotting of text
import seaborn as sns

Global seed set to 0
/home/jovyan/my-conda-envs/WE/lib/python3.9/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/home/jovyan/my-conda-envs/WE/lib/python3.9/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


In [3]:
# Code provided by Vitalii

def read_10x_output(smp_list, metadata=None, type = 'raw'):
    import os
    
    #Writing output from separate samples, processed using CellRanger, into a dictionary of Scanpy objects:
    ad = {}

    #Generate AnnData for each sample
    for sample_name in smp_list:
        path = sample_name
        for i in os.listdir(path):
            if type in i and 'h5' in i:
                file = i
        ad[sample_name] = sc.read_10x_h5(sample_name +'/'+file)
        ad[sample_name].var.rename(columns = {'gene_ids':'ENSEMBL'}, inplace = True)
        ad[sample_name].var['SYMBOL'] = ad[sample_name].var.index
        ad[sample_name].var.index = ad[sample_name].var['ENSEMBL']
        ad[sample_name].var.drop(columns=['ENSEMBL'], inplace=True)
        #ad[sample_name].var_names_make_unique() 
        
        
        sc.pp.calculate_qc_metrics(ad[sample_name], inplace=True)
        #ad[sample_name] = ad[sample_name][ad[sample_name].obs['total_counts'] > umi_filter, :]
        ad[sample_name].var['mt'] = [gene.startswith('mt-') 
                                     for gene in ad[sample_name].var['SYMBOL']]
        ad[sample_name].obs['mt_frac'] = (ad[sample_name][:, 
               ad[sample_name].var['mt'].tolist()].X.sum(1).A.squeeze() 
                                          / ad[sample_name].obs['total_counts'])
        
        ad[sample_name].obs['sample_id'] = sample_name
        ad[sample_name].obs['barcode'] = ad[sample_name].obs_names
        #ad[sample_name].obs_names = ad[sample_name].obs['barcode']+"_"+ad[sample_name].obs['sample_id']

    #Merge AnnData objects from all the samples together    
    from scipy.sparse import vstack
    stack = vstack([ad[x].X for x in smp_list]) # stack data
    adata = sc.AnnData(stack, var = ad[smp_list[0]].var)
    adata.obs = pd.concat([ad[x].obs for x in smp_list], axis = 0)

    if metadata is not None:
        #Add cleaned metadata to the Anndata.obs table
        # obs_merged = pd.merge(left = adata.obs, right = metadata, 
        #                      how = "left", left_on="sample", right_on="sample")
        # obs_merged.index = obs_merged['sample']+"_"+obs_merged['barcode']
        # print(obs_merged.index.equals(adata.obs.index))
        # adata.obs = obs_merged
        adata.obs[metadata.columns] = metadata.reindex(index=adata.obs['sample_id']).values
    for c in adata.obs.columns:
        adata.obs[c] = adata.obs[c].astype('str')
    adata.obs =  adata.obs.copy()

    return adata, ad

In [4]:
import os

arr = os.listdir()

In [5]:
new_samples= ['cellranger701_count_47447_HCA_BRA_F13774551_GRCh38-2020-A_m',
              'cellranger701_count_47447_HCA_BRA_F13774552_GRCh38-2020-A_m',
              'cellranger701_count_47447_HCA_BRA_F13774553_GRCh38-2020-A_m',
              'cellranger701_count_47447_HCA_BRA_F13774554_GRCh38-2020-A_m',
              'cellranger701_count_47447_HCA_BRA_F13774555_GRCh38-2020-A_m',
                'cellranger701_count_47447_HCA_BRA_F13774556_GRCh38-2020-A_m']

In [6]:
filter_data = [x for x in arr if
              any(y in x for y in new_samples)]

In [7]:
adata, ad_list = read_10x_output(
    smp_list=new_samples,
    
    type='filtered'
)

/home/jovyan/my-conda-envs/WE/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/jovyan/my-conda-envs/WE/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [8]:
adata

AnnData object with n_obs × n_vars = 122253 × 36601
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'mt_frac', 'sample_id', 'barcode'
    var: 'feature_types', 'genome', 'SYMBOL', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'mt'

# Preprocessing 

In [9]:
adata.obs

,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,mt_frac,sample_id,barcode
AAACAGCCAAGTCGCT-1,1864,7.5310163320779155,3349.0,8.116715,18.841445207524636,26.5750970438937,36.90653926545237,56.97223051657211,0.0,cellranger701_count_47447_HCA_BRA_F13774551_GR...,AAACAGCCAAGTCGCT-1
AAACAGCCAAGTGTCC-1,1305,7.174724309836376,1848.0,7.5224004,16.233766233766232,24.350649350649352,35.38961038961039,56.439393939393945,0.0,cellranger701_count_47447_HCA_BRA_F13774551_GR...,AAACAGCCAAGTGTCC-1
AAACAGCCAATGCCCG-1,1033,6.9411900550683745,1476.0,7.297768,20.663956639566393,29.200542005420054,42.75067750677507,63.888888888888886,0.0,cellranger701_count_47447_HCA_BRA_F13774551_GR...,AAACAGCCAATGCCCG-1
AAACAGCCACATGCTA-1,534,6.282266746896006,650.0,6.4785094,21.69230769230769,33.23076923076923,48.61538461538461,94.76923076923077,0.0,cellranger701_count_47447_HCA_BRA_F13774551_GR...,AAACAGCCACATGCTA-1
AAACAGCCAGCAGGTA-1,2034,7.618251097876695,3838.0,8.252967,21.18290776446066,28.999478895257948,38.587806149035956,56.17509119332986,0.0,cellranger701_count_47447_HCA_BRA_F13774551_GR...,AAACAGCCAGCAGGTA-1
...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGGTTGGTGAC-1,1299,7.170119543449628,2004.0,7.6033993,27.195608782435134,33.68263473053892,43.662674650698605,60.12974051896207,0.0,cellranger701_count_47447_HCA_BRA_F13774556_GR...,TTTGTTGGTTGGTGAC-1
TTTGTTGGTTTAACCC-1,2211,7.701652362642226,3550.0,8.174985,17.943661971830984,23.549295774647888,32.0,49.267605633802816,0.0,cellranger701_count_47447_HCA_BRA_F13774556_GR...,TTTGTTGGTTTAACCC-1
TTTGTTGGTTTACGTC-1,2571,7.85243908535751,4243.0,8.353262,17.11053499882159,22.837614895121376,30.73297195380627,46.476549611124206,0.0,cellranger701_count_47447_HCA_BRA_F13774556_GR...,TTTGTTGGTTTACGTC-1
TTTGTTGGTTTAGCTC-1,714,6.5722825426940075,919.0,6.8243737,23.39499455930359,33.18824809575626,44.069640914036995,76.71381936887923,0.0,cellranger701_count_47447_HCA_BRA_F13774556_GR...,TTTGTTGGTTTAGCTC-1


In [10]:
adata.var_names_make_unique()

In [11]:
adata.X.expm1().sum(axis = 1)


/home/jovyan/my-conda-envs/WE/lib/python3.9/site-packages/scipy/sparse/data.py:133: RuntimeWarning: overflow encountered in expm1
  result = op(self._deduped_data())


matrix([[          inf],
        [6.2351494e+27],
        [8.4383568e+26],
        ...,
        [          inf],
        [2.2937831e+27],
        [          inf]], dtype=float32)

# Add Metadata

In [12]:
adata.obs["Age"]="Multiplexed"

In [13]:
adata.obs["Organ"]="Meninges"

In [14]:
adata

AnnData object with n_obs × n_vars = 122253 × 36601
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'mt_frac', 'sample_id', 'barcode', 'Age', 'Organ'
    var: 'feature_types', 'genome', 'SYMBOL', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'mt'

In [15]:
adata.obs

,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,mt_frac,sample_id,barcode,Age,Organ
AAACAGCCAAGTCGCT-1,1864,7.5310163320779155,3349.0,8.116715,18.841445207524636,26.5750970438937,36.90653926545237,56.97223051657211,0.0,cellranger701_count_47447_HCA_BRA_F13774551_GR...,AAACAGCCAAGTCGCT-1,Multiplexed,Meninges
AAACAGCCAAGTGTCC-1,1305,7.174724309836376,1848.0,7.5224004,16.233766233766232,24.350649350649352,35.38961038961039,56.439393939393945,0.0,cellranger701_count_47447_HCA_BRA_F13774551_GR...,AAACAGCCAAGTGTCC-1,Multiplexed,Meninges
AAACAGCCAATGCCCG-1,1033,6.9411900550683745,1476.0,7.297768,20.663956639566393,29.200542005420054,42.75067750677507,63.888888888888886,0.0,cellranger701_count_47447_HCA_BRA_F13774551_GR...,AAACAGCCAATGCCCG-1,Multiplexed,Meninges
AAACAGCCACATGCTA-1,534,6.282266746896006,650.0,6.4785094,21.69230769230769,33.23076923076923,48.61538461538461,94.76923076923077,0.0,cellranger701_count_47447_HCA_BRA_F13774551_GR...,AAACAGCCACATGCTA-1,Multiplexed,Meninges
AAACAGCCAGCAGGTA-1,2034,7.618251097876695,3838.0,8.252967,21.18290776446066,28.999478895257948,38.587806149035956,56.17509119332986,0.0,cellranger701_count_47447_HCA_BRA_F13774551_GR...,AAACAGCCAGCAGGTA-1,Multiplexed,Meninges
...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGGTTGGTGAC-1,1299,7.170119543449628,2004.0,7.6033993,27.195608782435134,33.68263473053892,43.662674650698605,60.12974051896207,0.0,cellranger701_count_47447_HCA_BRA_F13774556_GR...,TTTGTTGGTTGGTGAC-1,Multiplexed,Meninges
TTTGTTGGTTTAACCC-1,2211,7.701652362642226,3550.0,8.174985,17.943661971830984,23.549295774647888,32.0,49.267605633802816,0.0,cellranger701_count_47447_HCA_BRA_F13774556_GR...,TTTGTTGGTTTAACCC-1,Multiplexed,Meninges
TTTGTTGGTTTACGTC-1,2571,7.85243908535751,4243.0,8.353262,17.11053499882159,22.837614895121376,30.73297195380627,46.476549611124206,0.0,cellranger701_count_47447_HCA_BRA_F13774556_GR...,TTTGTTGGTTTACGTC-1,Multiplexed,Meninges
TTTGTTGGTTTAGCTC-1,714,6.5722825426940075,919.0,6.8243737,23.39499455930359,33.18824809575626,44.069640914036995,76.71381936887923,0.0,cellranger701_count_47447_HCA_BRA_F13774556_GR...,TTTGTTGGTTTAGCTC-1,Multiplexed,Meninges
